In [1]:
import os
import ray
import ray.util
import random

In [2]:
from ray.util.client import ray as rayclient
if rayclient.is_connected():
    ray.util.disconnect()

headhost = os.environ['RAY_CLUSTER']
ray.util.connect('{ray_head}:10001'.format(ray_head=headhost))

{'num_clients': 1,
 'python_version': '3.6.8',
 'ray_version': '2.0.0.dev0',
 'ray_commit': '4357055305395ea309813851a539381bc7c32138',
 'protocol_version': '2020-02-22'}

In [3]:
# create some random data on [-1,1]X[-1,1]
# Here we are directly staging data, and so we invoke 'ray.put' instead of '@ray.remote'
def pi_random_points(n):
    # to submit a list as an object, must be wrapped in another list
    # so do that and then unpack it to get the object
    return ray.put([[(random.uniform(-1,1), random.uniform(-1,1)) for _ in range(n)]])[0]

n = 1000
data = pi_random_points(n)
data

ClientObjectRef(ffffffffffffffffffffffffffffffffffffffff0300000001000000)

In [4]:
# filter points inside the unit circle
@ray.remote
def pi_unit_circle(shard):
    return [p for p in shard if p[0]*p[0] + p[1]*p[1] <= 1]

in_circle = pi_unit_circle.remote(data)
in_circle

ClientObjectRef(883fd22a2c3722ddffffffffffffffffffffffff0300000001000000)

In [5]:
# count points in a partition
@ray.remote
def pi_count(p):
    return len(p)

counts = pi_count.remote(in_circle)
counts

ClientObjectRef(fd8c2e73df39afecffffffffffffffffffffffff0300000001000000)

In [6]:
c = ray.get(counts)
pi = 4 * c / n
print(f"pi= {pi}")

pi= 3.172


In [7]:
%%time
n = 100000
k = 10
random_points = [pi_random_points(n) for _ in range(k)]
in_circle = [pi_unit_circle.remote(p) for p in random_points]
circle_counts = [pi_count.remote(p) for p in in_circle]
partition_counts = ray.get(circle_counts)
c = sum(partition_counts)
pi = 4 * c / (n * k)
print(f"pi= {pi}")

pi= 3.144236
CPU times: user 3.87 s, sys: 161 ms, total: 4.03 s
Wall time: 11.8 s


In [8]:
@ray.remote
def count_in_circle_generator(n):
    def random_points(n):
        while n > 0:
            yield (random.uniform(-1,1), random.uniform(-1,1))
            n = n - 1
    c = 0
    for _ in (p for p in random_points(n) if p[0]*p[0] + p[1]*p[1] <= 1):
        c = c + 1
    return c

In [9]:
%%time
n = 100000
k = 10
x = [count_in_circle_generator.remote(n) for _ in range(k)]
c = sum(ray.get(x))
pi = 4 * c / (n * k)
print(f"pi= {pi}")

pi= 3.14086
CPU times: user 10.4 ms, sys: 2.98 ms, total: 13.4 ms
Wall time: 449 ms
